In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import os

In [ ]:
!pip install -U tensorboardcolab
from tensorboardcolab import *

  Created wheel for tensorboardcolab: filename=tensorboardcolab-0.0.22-cp37-none-any.whl size=3860 sha256=cc2d047e7fc2d6b790f9ea1415bfa0bf7257a95953ba1d39407085477a6b152b
  Stored in directory: /root/.cache/pip/wheels/c4/aa/a0/3aaf4f1a66adbdab9b7bdd4c96db8ada89eb7cd87200cfdd32
Successfully built tensorboardcolab


In [ ]:
tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://79beff493453.ngrok.io


In [ ]:
batch_size = 32
num_classes = 10
epochs = 20
data_augmentation = False
num_predictions = 20


In [ ]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 6s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
# 레이블을 원핫벡터로 만들어줌

In [ ]:
model = Sequential()
n_kernels = 32
# (배치사이즈,32 width,32 depth,3 channel) : input
# -> (32,32,32,3) mini-batch
model.add(Conv2D(n_kernels, (3, 3), padding='same',use_bias=False, kernel_initializer='he_normal'
                 ,input_shape=x_train.shape[1:]))
# (출력되는feature map의 갯수, convolution kernel size, padding same)
# (배치사이즈,width,depth,피처맵갯수n_kernels)
# (32묶음의 배치사이즈,32,32,32)
# ((3,3)*3채널+1바이어스)*n_kernels만큼 반복 + 바이어스개수 -> 파라미터 갯수
model.add(BatchNormalization())
model.add(Activation('relu'))
# (32,32,32,32)
model.add(Conv2D(32, (3, 3),use_bias=False, kernel_initializer='he_normal'))
model.add(BatchNormalization())
# (32,30,30,32)
model.add(Activation('relu'))
# (32,30,30,32)
model.add(MaxPooling2D(pool_size=(2, 2)))
# (32,15,15,32)
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same',use_bias=False, kernel_initializer='he_normal'))
model.add(BatchNormalization())
# (32,15,15,64)
model.add(Activation('relu'))
# (32,15,15,64)
model.add(Conv2D(64, (3, 3),use_bias=False, kernel_initializer='he_normal'))
model.add(BatchNormalization())
# (32,13,13,64)
model.add(Activation('relu'))
# (32,13,13,64)
model.add(MaxPooling2D(pool_size=(2, 2)))
# (32,6,6,64)
model.add(Dropout(0.25))
# (32,6,6,64)
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        864       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9216      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.Adam(lr=0.0001, decay=1e-6)

In [ ]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
if not data_augmentation:
    print('Not using data augmentation.')
    trained_model = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),callbacks=[TensorBoardColabCallback(tbc)],
                        workers=4)

Not using data augmentation.
Epoch 1/20
  81/1563 [>.............................] - ETA: 17s - loss: 1.4538 - accuracy: 0.4726

KeyboardInterrupt: ignored

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# BN을 적용해주면 정확도 살짝 상승

# 다시시도 zoom_range 0에서 0.2로 바꿔주고 rotation_range을 30도 주며
# data_augmentation=True로 하고 학습 시켜보면? 에러 뜸 이유는 모르겠음..

# 정확도를 높이기 위해 use_bias=False 추가에 kernel_initializer='he_normal'을 더해줌
# 성능이 사알짝. 올라감!

# VGG 같은 사용가능한 프리트래인 모델중에 엑스레이 사진 같은 흑백 이미지를 다루는 모델은 어떤게 있을까요?
# -> 답 사이트 : https://stanfordmlgroup.github.io/projects/chexnext/